In [28]:
import face_recognition
import cv2
import numpy as np
import os
import pickle
import time

# Initialize or load encodings and names
known_face_encodings = []
known_face_names = []
processed_files = set()

pickle_file_directory = r'C:\Users\adam.bloebaum\Desktop\face_encodings'
pickle_file_path = os.path.join(pickle_file_directory, 'face_encodings.pkl')

if os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'rb') as file:
        data = pickle.load(file)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
        processed_files = set(data['processed_files'])
    print(f"Loaded {len(known_face_encodings)} face encodings from file.")


training_directory = r'C:\Users\adam.bloebaum\Pictures\face_tracking_lib'
new_files_processed = False

if os.path.exists(training_directory):
    for filename in os.listdir(training_directory):
        file_path = os.path.join(training_directory, filename)
        
        if filename.endswith('.jpg') and filename not in processed_files:
            print(f"Processing new image: {filename}...")
            name_parts = filename.split('_')[:2]
            person_name = ' '.join(name_parts[::-1])
            
            image = face_recognition.load_image_file(file_path)
            encodings = face_recognition.face_encodings(image)

            if encodings:
                known_face_encodings.append(encodings[0])
                known_face_names.append(person_name)
                processed_files.add(filename)
                new_files_processed = True
            else:
                print(f"No faces found in {filename}, deleting...")
                os.remove(file_path)  # Delete the image file

# Check if the directory exists, if not, create it
if not os.path.exists(pickle_file_directory):
    os.makedirs(pickle_file_directory)

if new_files_processed or not os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'wb') as file:
        pickle.dump({
            'encodings': known_face_encodings,
            'names': known_face_names,
            'processed_files': list(processed_files)
        }, file)
    print("Face encodings updated and saved.")


# Initialize video capture for the internal webcam
cap = cv2.VideoCapture(0)  # Use 0 for internal webcam

# New code to track continuous recognition
recognized_timers = {}
identified_name = None
recognition_threshold = 3  # Time in seconds
process_this_frame = True

print("Starting video processing...")

# Loop over frames from the video file stream
while cap.isOpened():
    # Read the next frame from the file
    ret, frame = cap.read()
    # If the frame was not grabbed, then we have reached the end of the stream
    if not ret:
        print("Reached end of video.")
        break

    # Only process every other frame of video to save time
    if process_this_frame:
        print("Processing frame...")
        # Resize frame of video for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        small_frame = frame  # using original frame for now

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        print(f"Found {len(face_locations)} face(s) in this frame.")

        face_encodings = face_recognition.face_encodings(
            small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(
                known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
        print(f"Identified faces: {face_names}")

    # Switch to not process the next frame
    process_this_frame = not process_this_frame

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)  # Change color or thickness if desired
        
        # Set font scale and thickness
        font_scale = 0.5
        font_thickness = 1

        # Calculate text size for the name
        (text_width, text_height), baseline = cv2.getTextSize(name, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        text_offset_x = left
        text_offset_y = top - baseline - 3  # Adjust this value to position the text above the box

        # Draw rectangle and text for name
        cv2.rectangle(frame, (left, text_offset_y - text_height), (left + text_width, text_offset_y + baseline), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, name, (text_offset_x, text_offset_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
        
    cv2.imshow('Video', frame)  # Display the frame

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print("Releasing video objects...")
# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Loaded 19 face encodings from file.
Starting video processing...
Identified Adam Bloebaum for 3 seconds.
Releasing video objects...
Identified name: Adam Bloebaum


In [3]:
import face_recognition
import cv2
import numpy as np
import os
import pickle
import time

# Initialize or load encodings and names
known_face_encodings = []
known_face_names = []
processed_files = set()

pickle_file_directory = r'C:\Users\adam.bloebaum\Desktop\face_encodings'
pickle_file_path = os.path.join(pickle_file_directory, 'face_encodings.pkl')

# Load existing face encodings if available
if os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'rb') as file:
        data = pickle.load(file)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
        processed_files = set(data['processed_files'])
    print(f"Loaded {len(known_face_encodings)} face encodings from file.")

# Process new images and update encodings
training_directory = r'C:\Users\adam.bloebaum\Pictures\face_tracking_lib'
new_files_processed = False

if os.path.exists(training_directory):
    for filename in os.listdir(training_directory):
        file_path = os.path.join(training_directory, filename)
        
        if filename.endswith('.jpg') and filename not in processed_files:
            print(f"Processing new image: {filename}...")
            name_parts = filename.split('_')[:2]
            person_name = ' '.join(name_parts[::-1])
            
            image = face_recognition.load_image_file(file_path)
            encodings = face_recognition.face_encodings(image)

            if encodings:
                known_face_encodings.append(encodings[0])
                known_face_names.append(person_name)
                processed_files.add(filename)
                new_files_processed = True
            else:
                print(f"No faces found in {filename}, deleting...")
                os.remove(file_path)  # Delete the image file

# Save the updated encodings
if new_files_processed or not os.path.exists(pickle_file_path):
    if not os.path.exists(pickle_file_directory):
        os.makedirs(pickle_file_directory)
    
    with open(pickle_file_path, 'wb') as file:
        pickle.dump({
            'encodings': known_face_encodings,
            'names': known_face_names,
            'processed_files': list(processed_files)
        }, file)
    print("Face encodings updated and saved.")

# Initialize video capture for the internal webcam
cap = cv2.VideoCapture(0)  # Use 0 for internal webcam

recognized_timers = {}
identified_name = None
recognition_threshold = 3  # Time in seconds for continuous recognition
process_this_frame = True

print("Starting video processing...")

pause_message_displayed = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Only process every other frame of video to save time
    if process_this_frame:
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        small_frame = frame  # using original frame for now

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

        # Check continuous recognition
        current_time = time.time()
        for name in face_names:
            if name != "Unknown":
                if name not in recognized_timers:
                    recognized_timers[name] = current_time
                elif current_time - recognized_timers[name] >= recognition_threshold:
                    identified_name = name
                    print(f"Identified {name} for 3 seconds.")
                    break
            else:
                recognized_timers.pop(name, None)  # Remove unrecognized faces

        if identified_name:
            break  # If a name was identified for 3 seconds, break from the main loop

    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)  # Change color or thickness if desired
        
        # Set font scale and thickness
        font_scale = 0.5
        font_thickness = 1

        # Calculate text size for the name
        (text_width, text_height), baseline = cv2.getTextSize(name, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        text_offset_x = left
        text_offset_y = top - baseline - 3  # Adjust this value to position the text above the box

        # Draw rectangle and text for name
        cv2.rectangle(frame, (left, text_offset_y - text_height), (left + text_width, text_offset_y + baseline), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, name, (text_offset_x, text_offset_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
        
    process_this_frame = not process_this_frame

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
print("Releasing video objects...")
cap.release()
cv2.destroyAllWindows()

# Output the identified name
if identified_name:
    print(f"Identified name: {identified_name}")
else:
    print("No name identified for 3 continuous seconds.")


Loaded 19 face encodings from file.
Starting video processing...
Identified Adam Bloebaum for 3 seconds.
Releasing video objects...
Identified name: Adam Bloebaum


In [8]:
import face_recognition
import cv2
import numpy as np
import os
import pickle
import time

# Initialize or load encodings and names
known_face_encodings = []
known_face_names = []
processed_files = set()

pickle_file_directory = r'C:\Users\adam.bloebaum\Desktop\face_encodings'
pickle_file_path = os.path.join(pickle_file_directory, 'face_encodings.pkl')

# Load existing face encodings if available
if os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'rb') as file:
        data = pickle.load(file)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
        processed_files = set(data['processed_files'])
    print(f"Loaded {len(known_face_encodings)} face encodings from file.")

# Process new images and update encodings
training_directory = r'C:\Users\adam.bloebaum\Pictures\face_tracking_lib'
new_files_processed = False

if os.path.exists(training_directory):
    for filename in os.listdir(training_directory):
        file_path = os.path.join(training_directory, filename)
        
        if filename.endswith('.jpg') and filename not in processed_files:
            print(f"Processing new image: {filename}...")
            name_parts = filename.split('_')[:2]
            person_name = ' '.join(name_parts[::-1])
            
            image = face_recognition.load_image_file(file_path)
            encodings = face_recognition.face_encodings(image)

            if encodings:
                known_face_encodings.append(encodings[0])
                known_face_names.append(person_name)
                processed_files.add(filename)
                new_files_processed = True
            else:
                print(f"No faces found in {filename}, deleting...")
                os.remove(file_path)  # Delete the image file

# Save the updated encodings
if new_files_processed or not os.path.exists(pickle_file_path):
    if not os.path.exists(pickle_file_directory):
        os.makedirs(pickle_file_directory)
    
    with open(pickle_file_path, 'wb') as file:
        pickle.dump({
            'encodings': known_face_encodings,
            'names': known_face_names,
            'processed_files': list(processed_files)
        }, file)
    print("Face encodings updated and saved.")

# Initialize video capture for the internal webcam
cap = cv2.VideoCapture(0)  # Use 0 for internal webcam

recognized_timers = {}
identified_name = None
recognition_threshold = 3  # Time in seconds for continuous recognition
process_this_frame = True
pause_streaming = False
last_identified_name = None

print("Starting video processing...")

pause_message_displayed = False

while cap.isOpened():
    # Only read frame if not paused
    if not pause_streaming:
        ret, frame = cap.read()
        if not ret:
            break

    # Only process every other frame of video to save time
    if process_this_frame and not pause_streaming:
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        small_frame = frame  # using original frame for now

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

        # Check continuous recognition and pause logic
        current_time = time.time()
        for name in face_names:
            if name != "Unknown" and name != last_identified_name:
                if name not in recognized_timers:
                    recognized_timers[name] = current_time
                elif current_time - recognized_timers[name] >= recognition_threshold:
                    pause_streaming = True
                    last_identified_name = name
                    print(f"Identified {name} successfully.")
                    break
            else:
                recognized_timers.pop(name, None)  # Remove unrecognized or old faces

    # Draw rectangles and names on the frame
    if not pause_streaming:
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Draw a rectangle around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)  # Change color or thickness if desired
            
            # Set font scale and thickness
            font_scale = 0.5
            font_thickness = 1

            # Calculate text size for the name
            (text_width, text_height), baseline = cv2.getTextSize(name, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
            text_offset_x = left
            text_offset_y = top - baseline - 3  # Adjust this value to position the text above the box

            # Draw rectangle and text for name
            cv2.rectangle(frame, (left, text_offset_y - text_height), (left + text_width, text_offset_y + baseline), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (text_offset_x, text_offset_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
        
    process_this_frame = not process_this_frame

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Pause and resume logic
    if pause_streaming:
        # Display the pause message only once
        if not pause_message_displayed:
            print("Stream paused. Looking for a different face to resume...")
            pause_message_displayed = True

        # Capture a frame to check for a different face
        ret, frame = cap.read()
        if ret:
            small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
            small_frame = frame
            face_locations = face_recognition.face_locations(small_frame)
            face_encodings = face_recognition.face_encodings(small_frame, face_locations)

            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
                name = "Unknown"
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                if name != "Unknown" and name != last_identified_name:
                    print(f"Resuming on new face: {name}")
                    pause_streaming = False
                    recognized_timers.clear()

        if not pause_streaming:
            pause_message_displayed = False  # Reset the flag when streaming resumes


    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
print("Releasing video objects...")
cap.release()
cv2.destroyAllWindows()

# Output the identified name
if last_identified_name:
    print(f"Identified name: {last_identified_name}")
else:
    print("No name identified for 3 continuous seconds.")


Loaded 19 face encodings from file.
Starting video processing...
Identified Adam Bloebaum successfully.
Stream paused. Looking for a different face to resume...
Releasing video objects...
Identified name: Adam Bloebaum
